# Imports

In [3]:
import logging

import pandas as pd

import helpers.hdatetime as hdateti
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hpandas as hpandas
import helpers.hprint as hprint
import im_v2.ccxt.data.extract.extractor as imvcdexex
import im_v2.common.data.client.im_raw_data_client as imvcdcimrdc
import im_v2.common.data.qa.dataset_validator as imvcdqdava
import im_v2.common.data.qa.qa_check as imvcdqqach
import im_v2.common.universe as ivcu
import im_v2.common.universe.universe as imvcounun

%load_ext autoreload
%autoreload 2

/app/core/signal_processing/decorrelation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-05def9c6-f47b-482e-b9a2-57c11576b7d1.json'
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.4.3' != container_version='1.4.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='CmTask3917_Onboard_CCXT_Binance_historical_trades'
  hash='e1f5a9f33'
  # Last commits:
    * e1f5a9f33 vlady    CmTask3917: minor changes by comments                             (51 minutes ago) Tue Mar 7 18:51:59 2023  (HEAD -> CmTask3917_Onboard_CCXT_Binance_historical_trades)
    * 470cf5c60 vlady    CmTask3917: fix slow test                                         (   3 hours ago) Tue Mar 7 16:18:33 2023  (origin/CmTask

# Bulk download script

## Airflow first test run

In [13]:
%%bash
/app/im_v2/common/data/extract/download_bulk.py \
    --download_mode 'periodic_daily' \
    --downloading_entity 'airflow' \
    --action_tag 'downloaded_1min' \
    --vendor 'ccxt' \
    --start_timestamp '2023-02-05T00:00:00+00:00' \
    --end_timestamp '2023-02-05T00:10:00+00:00' \
    --exchange_id 'binance' \
    --universe 'v7.3' \
    --aws_profile 'ck' \
    --data_type 'trades' \
    --data_format 'parquet' \
    --contract_type 'futures' \
    --s3_path 's3://cryptokaizen-data-test/'

INFO: > cmd='/app/im_v2/common/data/extract/download_bulk.py --download_mode periodic_daily --downloading_entity airflow --action_tag downloaded_1min --vendor ccxt --start_timestamp 2023-02-05T00:00:00+00:00 --end_timestamp 2023-02-05T00:10:00+00:00 --exchange_id binance --universe v7.3 --aws_profile ck --data_type trades --data_format parquet --contract_type futures --s3_path s3://cryptokaizen-data-test/'
report_memory_usage=False report_cpu_usage=False
INFO: Saving log to file '/app/im_v2/common/data/extract/download_bulk.py.log'
10:13:13 - INFO  dataset_schema_utils.py _get_dataset_schema_file_path:39 Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
10:13:13 - INFO  dataset_schema_utils.py get_dataset_schema:74          Loaded dataset schema version v3
10:13:23 - INFO  extract_utils.py download_historical_data:733          Saving the dataset into s3://cryptokaizen-data-test/v3/periodic_daily/airflow/downloaded_1min/parquet/trades/futures/v

In [85]:
%%bash
/app/im_v2/common/data/extract/download_bulk.py \
    --download_mode 'bulk' \
    --downloading_entity 'manual' \
    --action_tag 'downloaded_1min' \
    --vendor 'ccxt' \
    --start_timestamp '2023-03-01T00:00:00+00:00' \
    --end_timestamp '2023-03-01T01:00:00+00:00' \
    --exchange_id 'binance' \
    --universe 'v7.3' \
    --incremental \
    --aws_profile 'ck' \
    --data_type 'trades' \
    --data_format 'parquet' \
    --contract_type 'futures' \
    --s3_path 's3://cryptokaizen-data-test/'

INFO: > cmd='/app/im_v2/common/data/extract/download_bulk.py --download_mode bulk --downloading_entity manual --action_tag downloaded_1min --vendor ccxt --start_timestamp 2023-03-01T00:00:00+00:00 --end_timestamp 2023-03-01T01:00:00+00:00 --exchange_id binance --universe v7.3 --incremental --aws_profile ck --data_type trades --data_format parquet --contract_type futures --s3_path s3://cryptokaizen-data-test/'
report_memory_usage=False report_cpu_usage=False
INFO: Saving log to file '/app/im_v2/common/data/extract/download_bulk.py.log'
12:19:28 - INFO  dataset_schema_utils.py _get_dataset_schema_file_path:37 Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
12:19:28 - INFO  dataset_schema_utils.py get_dataset_schema:72          Loaded dataset schema version v3
12:20:26 - INFO  extract_utils.py download_historical_data:717          Saving the dataset into s3://cryptokaizen-data-test/v3/bulk/manual/downloaded_1min/parquet/trades/futures/v7_3/ccxt

## Manual QA - airflow download

In [8]:
signature = "periodic_daily.airflow.downloaded_1min.parquet.trades.futures.v7_3.ccxt.binance.v1_0_0"
reader = imvcdcimrdc.RawDataReader(signature, stage="test")
start_timestamp = pd.Timestamp("2023-03-07T18:00:00+00:00")
end_timestamp = pd.Timestamp("2023-03-07T19:00:00+00:00")

INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3


In [9]:
trades = reader.read_data(start_timestamp, end_timestamp)

INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3


In [10]:
trades.head()

,timestamp,symbol,side,price,amount,exchange_id,knowledge_timestamp,currency_pair,year,month,day
timestamp,,,,,,,,,,,
2023-03-07 18:51:00.519000+00:00,1678215060519,APE/USDT,sell,4.628,22.0,binance,2023-03-07 18:55:54.121942+00:00,APE_USDT,2023,3,7
2023-03-07 18:51:01.612000+00:00,1678215061612,APE/USDT,buy,4.629,6.0,binance,2023-03-07 18:55:54.121942+00:00,APE_USDT,2023,3,7
2023-03-07 18:51:01.836000+00:00,1678215061836,APE/USDT,sell,4.628,200.0,binance,2023-03-07 18:55:54.121942+00:00,APE_USDT,2023,3,7
2023-03-07 18:51:02.575000+00:00,1678215062575,APE/USDT,buy,4.629,25.0,binance,2023-03-07 18:55:54.121942+00:00,APE_USDT,2023,3,7
2023-03-07 18:51:02.850000+00:00,1678215062850,APE/USDT,buy,4.629,24.0,binance,2023-03-07 18:55:54.121942+00:00,APE_USDT,2023,3,7


In [15]:
trades_pairs = trades.symbol.str.replace("/", "_").unique()

In [12]:
vendor = "CCXT"
mode = "download"
exchange_id = "binance"
version = "v7.3"
universe = imvcounun.get_vendor_universe(vendor, mode, version=version)

In [16]:
~pd.Series(trades_pairs).isin(universe[exchange_id]).any()

False

## Manual QA - bulk download

In [98]:
signature = (
    "bulk.manual.downloaded_1min.parquet.trades.futures.v7_3.ccxt.binance.v1_0_0"
)
reader = imvcdcimrdc.RawDataReader(signature, stage="test")
start_timestamp = pd.Timestamp("2021-03-01T00:00:00+00:00")
end_timestamp = pd.Timestamp("2021-03-01T01:00:00+00:00")

INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3


In [99]:
trades = reader.read_data(start_timestamp, end_timestamp)

INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3


In [100]:
pairs_from_trades = trades.currency_pair.unique()

In [89]:
vendor = "CCXT"
mode = "download"
exchange_id = "binance"
version = "v7.3"
universe = imvcounun.get_vendor_universe(vendor, mode, version=version)

In [90]:
~pd.Series(pairs_from_trades).isin(universe[exchange_id]).any()

False

In [91]:
trades.timestamp.agg(["min", "max"]).apply(pd.Timestamp, unit="ms")

min   2023-03-01 00:00:00.045
max   2023-03-01 00:59:59.971
Name: timestamp, dtype: datetime64[ns]

In [95]:
trades.isna().any().all()

False

## Manual QA - periodic daily

In [107]:
signature = "periodic_daily.airflow.downloaded_1min.parquet.trades.futures.v7_3.ccxt.binance.v1_0_0"
reader = imvcdcimrdc.RawDataReader(signature, stage="test")
start_timestamp = pd.Timestamp("2023-03-03T17:00:00+00:00")
end_timestamp = pd.Timestamp("2023-03-03T17:12:00+00:00")

INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3


In [108]:
trades = reader.read_data(start_timestamp, end_timestamp)

INFO  Loading dataset schema file: /app/data_schema/dataset_schema_versions/dataset_schema_v3.json
INFO  Loaded dataset schema version v3


In [109]:
trades

,timestamp,price,size,is_buyer_maker,exchange_id,knowledge_timestamp,currency_pair,year,month
timestamp,,,,,,,,,
2023-03-03 17:09:00.061000+00:00,1677863340061,4.8550,67.0,0,binance,2023-03-03 17:14:58.183456+00:00,APE_USDT,2023,3
2023-03-03 17:09:00.850000+00:00,1677863340850,4.8550,8.0,0,binance,2023-03-03 17:14:58.183456+00:00,APE_USDT,2023,3
2023-03-03 17:09:01.049000+00:00,1677863341049,4.8550,7.0,0,binance,2023-03-03 17:14:58.183456+00:00,APE_USDT,2023,3
2023-03-03 17:09:01.328000+00:00,1677863341328,4.8550,306.0,1,binance,2023-03-03 17:14:58.183456+00:00,APE_USDT,2023,3
2023-03-03 17:09:01.509000+00:00,1677863341509,4.8540,56.0,0,binance,2023-03-03 17:14:58.183456+00:00,APE_USDT,2023,3
...,...,...,...,...,...,...,...,...,...
2023-03-03 17:08:59.510000+00:00,1677863339510,0.3673,2700.0,0,binance,2023-03-03 17:11:58.547293+00:00,XRP_USDT,2023,3
2023-03-03 17:08:59.616000+00:00,1677863339616,0.3673,3600.0,0,binance,2023-03-03 17:11:58.547293+00:00,XRP_USDT,2023,3
2023-03-03 17:08:59.758000+00:00,1677863339758,0.3673,2700.0,0,binance,2023-03-03 17:11:58.547293+00:00,XRP_USDT,2023,3


In [111]:
trades.isna().any().all()

False

In [112]:
pairs_from_trades = trades.currency_pair.unique()

In [113]:
~pd.Series(pairs_from_trades).isin(universe[exchange_id]).any()

False

In [114]:
trades.timestamp.agg(["min", "max"]).apply(pd.Timestamp, unit="ms")

min   2023-03-03 17:00:00.051
max   2023-03-03 17:11:59.967
Name: timestamp, dtype: datetime64[ns]

# Bare CCXT extractor test

In [42]:
import ccxt

extractor = ccxt.binance()

In [43]:
start_timestamp = pd.Timestamp("2021-01-01T00:00:00+00:00")
start_timestamp = hdateti.convert_timestamp_to_unix_epoch(start_timestamp)
start_timestamp = 1614556800000
currency_pair = "ETH/BUSD"
limit = 500
data = extractor.fetch_trades(
    currency_pair,
    since=start_timestamp,
    limit=limit,
)

In [44]:
start_timestamp

1614556800000

In [45]:
len(data)

500

# CCXT extractor test

In [29]:
import im_v2.ccxt.data.extract.extractor as imvcdexex

exchange_id = "binance"
contract_type = "futures"
ccxt_extractor = imvcdexex.CcxtExtractor(exchange_id, contract_type)

In [32]:
data_type = "trades"
converted_currency_pair = currency_pair
start_timestamp = pd.Timestamp("2021-03-01T00:00:00+00:00")
end_timestamp = pd.Timestamp("2021-03-01T01:00:00+00:00")
data = ccxt_extractor.download_data(
    data_type,
    exchange_id,
    converted_currency_pair,
    start_timestamp=start_timestamp,
    end_timestamp=end_timestamp,
    depth=10,
)

RuntimeError: No trades were fetched with the given parameters: currency_pair=ETH/BUSD, start_timestamp=1614556800000, end_timestamp=1614560400000

In [ ]:
data